### MRMEGA

#### Post QC on Million Veteran Program GWAS included filtering out unrealistic standard errors (SE)
#### Filter out 0.99>MAF>0.01 and unrealistic SE variants

In [ ]:
zcat GCST90477503.tsv.gz | awk 'BEGIN{FS=OFS="\t"} NR==1{print;next} $5!="#NA" && $10!="#NA" && $11!="#NA" && ($5+0>0) && ($10+0>0) && ($11+0>0) && ($11+0 < $5+0 && $5+0 < $10+0) && ($5+0>=0.01 && $5+0<=5) && ($10+0>=0.01 && $10+0<=5) && ($11+0>=0.01 && $11+0<5) && ($7=="" || ($7+0>=0.009 && $7+0<=0.991)) {print}' | gzip -c > GCST90477503_filt.gz

zcat GCST90475828.tsv.gz | awk 'BEGIN{FS=OFS="\t"} NR==1{print;next} $5!="#NA" && $10!="#NA" && $11!="#NA" && ($5+0>0) && ($10+0>0) && ($11+0>0) && ($11+0 < $5+0 && $5+0 < $10+0) && ($5+0>=0.01 && $5+0<=5) && ($10+0>=0.01 && $10+0<=5) && ($11+0>=0.01 && $11+0<5) && ($7=="" || ($7+0>=0.009 && $7+0<=0.991)) {print}' | gzip -c > GCST90475828_filt.gz

zcat GCST90475829.tsv.gz | awk 'BEGIN{FS=OFS="\t"} NR==1{print;next} $5!="#NA" && $10!="#NA" && $11!="#NA" && ($5+0>0) && ($10+0>0) && ($11+0>0) && ($11+0 < $5+0 && $5+0 < $10+0) && ($5+0>=0.01 && $5+0<=5) && ($10+0>=0.01 && $10+0<=5) && ($11+0>=0.01 && $11+0<5) && ($7=="" || ($7+0>=0.009 && $7+0<=0.991)) {print}' | gzip -c > GCST90475829_filt.gz

#### Annotate markername and EAF in MVP GWAS data

In [ ]:
awk -F'\t' 'NR==FNR { if(NR>1) m[$1]=$2; next }
            FNR==1   { print $0 FS "EAF"; next }
                     { print $0 FS ( ($1 in m)?m[$1]:"NA") }'   <(zcat GCST90477503_filt.gz | cut -f7,9 | awk -F'\t' '{print $2"\t"$1}')   AFR_GCST90477503_filt_BETASE_markername | gzip -c > AFR_GCST90477503_filt_BETASE_markername_EAFforMEGA.gz

awk -F'\t' 'NR==FNR { if(NR>1) m[$1]=$2; next }
            FNR==1   { print $0 FS "EAF"; next }
                     { print $0 FS ( ($1 in m)?m[$1]:"NA") }'   <(zcat GCST90475828_filt.gz | cut -f7,9 | awk -F'\t' '{print $2"\t"$1}')   GCST90475828_filt_BETASE_pos_markeradd | gzip -c > GCST90475828_filt_BETASE_pos_markeradd_amr_EAF_formrmega.gz

awk -F'\t' 'NR==FNR { if(NR>1) m[$1]=$2; next }
            FNR==1   { print $0 FS "EAF"; next }
                     { print $0 FS ( ($1 in m)?m[$1]:"NA") }'   <(zcat GCST90475829_filt.gz | cut -f7,9 | awk -F'\t' '{print $2"\t"$1}')  EUR_GCST90475829_filt_filt_BETASE_markername | gzip -c > EUR_GCST90475829_filt_filt_BETASE_markername_eaf_formrmega.gz

#### MRMEGA input file

In [1]:
cat MR-MEGA.in

rls_ukb_eur_sumstat_ci.gz
allofus_afr_sumstat.gz
allofus_amr_sumstat.gz
allofus_eur_sumstat.gz
cartagene_eur_sumstat_cols.gz
clsa_eur_sumstat.gz
AFR_GCST90477503_filt_BETASE_markername_EAFforMEGA_pos.gz
GCST90475828_filt_BETASE_pos_markeradd_amr_EAF_formrmega_pos.gz
EUR_GCST90475829_filt_filt_BETASE_markername_eaf_formrmega_pos.gz


#### Running MRMEGA

In [ ]:
./MR-MEGA -i MR-MEGA.in --pc 3 -o AUGS2025_MRMEGA_pc

### PLINK RANDOM EFFECT

In [ ]:
plink --meta-analysis allofus_afr allofus_amr allofus_eur cartagene_eur clsa_eur mva_afr mva_amr mva_eur ukb_forplink + logscale study qt \
--out RLS_multirandomeff

--meta-analysis: 13506491 variants processed; results written to
RLS_multirandomeff.meta

awk -F'\t' 'BEGIN{OFS="\t"} NR==1 || $6 >= 3' RLS_multirandomeff.meta > RLS_multirandomeff.meta_min3study
gzip -c RLS_multirandomeff.meta_min3study > RLS_multirandomeff.meta_min3study_gwaslab.gz